In [1]:
import json
import time 
from kafka import KafkaProducer

In [2]:
def json_serializer(data):
    return json.dumps(data).encode('utf-8')

In [3]:
server = 'localhost:9092'

In [4]:
producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

In [5]:
producer.bootstrap_connected()

True

In [6]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

t_before_flush = time.time()
producer.flush()

t1 = time.time()
print(f'sending_take_time {(t_before_flush - t0):.2f} seconds')
print(f'flush_take_time {(t1 - t_before_flush):.2f} seconds')
print(f'took {(t1 - t0):.2f} seconds')

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
sending_take_time 0.51 seconds
flush_take_time 0.00 seconds
took 0.51 seconds


In [7]:
import pandas as pd

In [8]:
desired_columns = ["lpep_pickup_datetime", "lpep_dropoff_datetime",
                  "PULocationID", "DOLocationID",
                  "passenger_count", "trip_distance",
                  "tip_amount"]

In [9]:
df_green = pd.read_csv('green_tripdata_2019-10.csv.gz', usecols=desired_columns)

In [10]:
t0 = time.time()
topic_name = 'green-trips'
for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    producer.send(topic_name, value=row_dict)

producer.flush()
t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

took 46.16 seconds


In [11]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

:: loading settings :: url = jar:file:/home/abdol/spark/spark-3.3.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/abdol/.ivy2/cache
The jars for the packages stored in: /home/abdol/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e8de850b-f13b-405c-8ac8-b16519f7ff78;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.2 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolu

24/03/29 15:38:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [12]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [13]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

24/03/29 15:38:50 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-81f81811-2f62-4622-8dc3-1d35caf7b1cb. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/29 15:38:50 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Row(key=None, value=bytearray(b'{"lpep_pickup_datetime": "2019-10-01 00:26:02", "lpep_dropoff_datetime": "2019-10-01 00:39:58", "PULocationID": 112, "DOLocationID": 196, "passenger_count": 1.0, "trip_distance": 5.88, "tip_amount": 0.0}'), topic='green-trips', partition=0, offset=0, timestamp=datetime.datetime(2024, 3, 29, 15, 36, 0, 441000), timestampType=0)


In [14]:
query.stop()

In [15]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [16]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [18]:
from pyspark.sql.functions import current_timestamp, window, col
from pyspark.sql.types import IntegerType

In [19]:
green_stream = green_stream.withColumn("timestamp", current_timestamp())

In [20]:
popular_destinations = green_stream \
    .groupBy(window(col("timestamp"), "5 minutes"), col("DOLocationID")) \
    .count() \
    .orderBy("count", ascending=False)

In [21]:
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

24/03/29 15:42:08 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-84cd1709-7f48-48b1-8708-fde9d5101e8e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/29 15:42:08 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-29 15:40:00, 2024-03-29 15:45:00}|74          |17741|
|{2024-03-29 15:40:00, 2024-03-29 15:45:00}|42          |15942|
|{2024-03-29 15:40:00, 2024-03-29 15:45:00}|41          |14061|
|{2024-03-29 15:40:00, 2024-03-29 15:45:00}|75          |12840|
|{2024-03-29 15:40:00, 2024-03-29 15:45:00}|129         |11930|
|{2024-03-29 15:40:00, 2024-03-29 15:45:00}|7           |11533|
|{2024-03-29 15:40:00, 2024-03-29 15:45:00}|166         |10845|
|{2024-03-29 15:40:00, 2024-03-29 15:45:00}|236         |7913 |
|{2024-03-29 15:40:00, 2024-03-29 15:45:00}|223         |7542 |
|{2024-03-29 15:40:00, 2024-03-29 15:45:00}|238         |7318 |
|{2024-03-29 15:40:00, 2024-03-29 15:45:00}|82          |7292 |
|{2024-

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/abdol/spark/spark-3.3.2-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/abdol/spark/spark-3.3.2-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/home/abdol/anaconda3/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [22]:
query.stop()

In [24]:
spark.stop()